# 토큰화 실행
- 미리 구축해둔 vocab.txt를 사용하겠다


In [1]:
!pip install ratsnlp

     |████████████████████████████████| 42 kB 358 kB/s 
     |████████████████████████████████| 57 kB 4.0 MB/s 
     |████████████████████████████████| 806 kB 10.5 MB/s 
     |████████████████████████████████| 2.8 MB 25.0 MB/s 
     |████████████████████████████████| 397 kB 38.6 MB/s 
     |████████████████████████████████| 636 kB 44.2 MB/s 
     |████████████████████████████████| 829 kB 39.0 MB/s 
     |████████████████████████████████| 133 kB 53.6 MB/s 
     |████████████████████████████████| 67 kB 4.7 MB/s 
     |████████████████████████████████| 895 kB 55.9 MB/s 
     |████████████████████████████████| 3.3 MB 39.0 MB/s 
     |████████████████████████████████| 1.1 MB 56.9 MB/s 
     |████████████████████████████████| 96 kB 4.8 MB/s 
     |████████████████████████████████| 271 kB 45.4 MB/s 
     |████████████████████████████████| 144 kB 49.7 MB/s 
     |████████████████████████████████| 94 kB 2.9 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sh

# 구글 드라이브 연동

- 앞의 예제에서 구축한 어휘 집합을 활용하겠다.

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


## GPT 입력값 만들기

- GPT 모델 입력값을 만들기 위해서 미리 만들어둔 BBPE 어휘집합 구축 결과를 이용하겠다. (`vocab.json`, `merges.txt`)
- 아래를 수행해 이미 만들어 놓은 BBPE 어휘집합을 포함한 GPT 토크나이저를 `tokenizer_gpt`라는 변수로 선언한다

In [3]:
from transformers import GPT2Tokenizer
tokenizer_gpt = GPT2Tokenizer.from_pretrained("/gdrive/My Drive/nlpbook/bbpe")
tokenizer_gpt.pad_token = "[PAD]"

file /gdrive/My Drive/nlpbook/bbpe/config.json not found


- 문장 세 개 토큰화

In [4]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_gpt.tokenize(sentence) for sentence in sentences]

In [5]:
tokenized_sentences

[['ìķĦ', 'ĠëįĶë¹Ļ', '..', 'Ġì§Ħì§ľ', 'Ġì§ľì¦ĿëĤĺ', 'ëĦ¤ìļĶ', 'Ġëª©ìĨĮë¦¬'],
 ['íĿł',
  '...',
  'íı¬ìĬ¤íĦ°',
  'ë³´ê³ł',
  'Ġì´ĪëĶ©',
  'ìĺģíĻĶ',
  'ì¤Ħ',
  '....',
  'ìĺ¤ë²Ħ',
  'ìĹ°ê¸°',
  'ì¡°ì°¨',
  'Ġê°Ģë³į',
  'ì§Ģ',
  'ĠìķĬ',
  'êµ¬ëĤĺ'],
 ['ë³Ħë£¨', 'Ġìĺ', 'Ģëĭ¤', '..']]

- 배치의 크기가 3이라고 가정하고 배치의 입력값을 생성
- 문자 길이를 12로 제한

In [6]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
batch_inputs = tokenizer_gpt(
    sentences,
    padding="max_length",
    max_length=12,
    truncation=True,
)

`batch_inputs`의 내용 확인

In [7]:
batch_inputs.keys()

dict_keys(['input_ids', 'attention_mask'])

In [8]:
batch_inputs['input_ids']

[[334, 2338, 263, 581, 4055, 464, 3808, 0, 0, 0, 0, 0],
 [3693, 336, 2876, 758, 2883, 356, 806, 422, 9875, 875, 2960, 7292],
 [4957, 451, 3653, 263, 0, 0, 0, 0, 0, 0, 0, 0]]

In [9]:
batch_inputs['attention_mask']

[[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

## BERT 입력값 만들기

- 자신의 구글 드라이브 경로(`/gdrive/My Drive/nlpbook/wordpiece`)에 워드피스 어휘집합 구축 결과(`vocab.txt`)를 이용하겠다
- BERT 토크나이저를 `tokenizer_bert`라는 변수로 선언

In [10]:
from transformers import BertTokenizer
tokenizer_bert = BertTokenizer.from_pretrained(
    "/gdrive/My Drive/nlpbook/wordpiece", 
    do_lower_case=False,
)

file /gdrive/My Drive/nlpbook/wordpiece/config.json not found


In [11]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_bert.tokenize(sentence) for sentence in sentences]

In [12]:
tokenized_sentences

[['아', '더빙', '.', '.', '진짜', '짜증나', '##네요', '목소리'],
 ['흠',
  '.',
  '.',
  '.',
  '포스터',
  '##보고',
  '초딩',
  '##영화',
  '##줄',
  '.',
  '.',
  '.',
  '.',
  '오버',
  '##연기',
  '##조차',
  '가볍',
  '##지',
  '않',
  '##구나'],
 ['별루', '였다', '.', '.']]

- 배치 크기가 3이라고 가정

In [13]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
batch_inputs = tokenizer_bert(
    sentences,
    padding="max_length",
    max_length=12,
    truncation=True,
)

In [14]:
batch_inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

- 인덱스 2는 [CLS], 3은 [SEP] 토큰을 나타낸다

In [15]:
batch_inputs['input_ids']

[[2, 620, 2631, 16, 16, 1993, 3678, 1990, 3323, 3, 0, 0],
 [2, 997, 16, 16, 16, 2609, 2045, 2796, 1981, 1044, 16, 3],
 [2, 3274, 9508, 16, 16, 3, 0, 0, 0, 0, 0, 0]]

In [16]:
batch_inputs['attention_mask']

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]

- 문장을 하나씩만 넣었으므로 세그먼트는 모두 0이다.

In [17]:
batch_inputs['token_type_ids']

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]